In [0]:
print("Healthy")

In [0]:
from re import search
from sqlalchemy import create_engine
import duckdb

import pandas as pd 
import numpy as np 
import mysql.connector


pd.set_option('display.max_columns', None)


import geopy
from geopy.geocoders import ArcGIS, Nominatim

import warnings
warnings.filterwarnings('ignore')

import json
import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
     host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)


host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
engine = conn

pd.set_option('display.float_format', lambda x: '%.2f' % x)

from datetime import datetime, timedelta
today = datetime.today().strftime('%Y-%m-%d')
yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d')
print(today)
print(yesterday)


last_2_wks = datetime.today() - timedelta(days = 14)
last_2_wks = last_2_wks.strftime('%Y-%m-%d')
print('------------------------------------')
print(last_2_wks)

print('\n')
now = datetime.today().strftime('%Y-%m-%d %H:%M:%S')

last_30_mins = (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d %H:%M:%S')
trunc_last_30_mins = (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d %H:%M')
print(last_30_mins, 'to', now)

In [0]:
# all produced cards 
cvd = pd.read_sql_query(f'''
                select
                ID as id, 
                PAN_DATA as pan_data, 
                MASKED_PAN as masked_pan, 
                EXPIRY_DATE as expiry_date_code, 
                CUSTOMER_DATA_1,
                CUSTOMER_DATA_2, 
                C_STATUS as card_status, 
                DATE_CREATED as created_date, 
               ACCOUNT_NO as account_no,
               ACCOUNT_ID as account_id, 
               CLIENT_ID as client_id, 
               MOBILE_NO as mobile_no, 
               PAN_TYPE as pan_type
                from CM_VERVE_DETAILS
                ''', Dumpdb)
details = pd.read_sql(f'''
                select
                client_id,
                client_name,
                email_address
                from dwh_all_clients dac
                where client_id in ({','.join([str(x) for x in cvd['client_id'].tolist()])})
              ''', conn)

details['client_id'] = details['client_id'].astype(str)

final = pd.merge(cvd, details, on = 'client_id', how ='left')

final

In [0]:
#final['expiry_date'] = ['20'+code[:2]+'-'+code[2:]+'-'+'01' for code in final['expiry_date_code']]
#final['expiry_date'] = [datetime.strptime(date, '%Y-%m-%d') for date in final['expiry_date']]
final

In [0]:
#inval = final[final['expiry_date'] == '2000-01']
inval = final
inval.to_excel("/Workspace/ReportDump/ETL_JOBS_FILES/Invalid card expiry dates.xlsx", index = False)

In [0]:
final['expiry_date_code'].dtype

In [0]:
import os 
os.getcwd()

Virtual and Physical requests/produced cards from January to June 2023.

Should have count and value with delivery address of physical Cards

Failure report for Mobile and Cards. response code and error on column, while Montly volume and value on the rows

In [0]:
%%time
final.to_sql('dwh_card_produced', engine, index = False, if_exists = 'replace', chunksize= 10000, method = 'multi')


Report of all card requests via the VBank app from November 2022 till date
 

Request Date

Balance on Request

Name

Email

Phone

Account Number

Delivery Address

 

Report of all accounts with POS on MIFOs as at date
client_name

account_no

pos_type

terminal_id

PRODUCT NAME

MSC_percent

POS Configuration Date



In [0]:
cdrqst = pd.read_sql_query(f'''
                select
                ID as id, 
                NAME as client_name, 
                EMAIL as email_address, 
                PHONE as mobile_no, 
                ACCOUNT_NUMBER as account_no, 
                ACCOUNT_BALANCE as account_balance,
                DELIVERY_ADDRESS as delivery_address, 
                DATE_CREATED as date_requested, 
                RESOURCE_ID as transaction_id
                from CARD_REQUEST
                -- where left(date_created, 7) = '2023-07'
                ''', Dumpdb)

cdrqst

In [0]:
# cdrqst[(cdrqst['transaction_id'].notna()) & (cdrqst['transaction_id'] != '')]

In [0]:
tid = cdrqst[(cdrqst['transaction_id'].notna()) & (cdrqst['transaction_id'] != '')]['transaction_id']
chrg = pd.read_sql_query(f'''
                        select
                        client_id,
                        transaction_id,
                        amount as amount_charged,
                        transaction_remarks
                        from dwh_all_transactions
                        where transaction_id in ({','.join([str(x) for x in tid])})
                        group by 1, 2, 3, 4
                        ''', conn)
chrg['transaction_id'] = chrg['transaction_id'].astype(str)

In [0]:
cdrqst = pd.merge(cdrqst, chrg, on = 'transaction_id', how = 'left')

conditions = [
    cdrqst['amount_charged'] <= 1000,
    cdrqst['amount_charged'] > 1000
]

choices = ['Virtual', 'Physical']

cdrqst['card_type'] = np.select(conditions, choices, default='Unclassified')
cdrqst

In [0]:
# with pd.ExcelWriter("C:/Users/babatunde.omotayo/Downloads/V_bank_tasks/misc folder/Billy/Virtual cards request details Jul-2023.xlsx") as writer:
#     vc.to_excel(writer, sheet_name = 'Sheet 1', index = False)

In [0]:
%%time
## drop existing card request table
q  = '''
DROP TABLE dwh_card_requests CASCADE;
'''
with engine.begin() as conn:     
    conn.execute(q)
print('dwh_card_requests dropped successfully!\n')


## replace table
cdrqst.to_sql('dwh_card_requests', engine, index = False, if_exists = 'replace', chunksize= 10000, method = 'multi')
print('dwh_card_requests replaced successfully!')

### Card Transactions

In [0]:
## initialize DB connection afresh
host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
engine = conn

In [0]:
## existing cards trxns 
ltr = pd.read_sql(f'''
                select
                max(id) as ltid
                from dwh_card_transactions dct
                ''', conn)

ltid = ltr['ltid'].unique()[0]
print(ltid)

In [0]:
%%time
ctrans = pd.read_sql(f'''
            select
            *
            from TM_VERVE_TRANSACTIONS 
            where id > {ltid}
            ''', Dumpdb)

ctrans = pd.DataFrame([row[1:] for row in ctrans.itertuples()], columns = [col.lower() for col in ctrans.columns])
ctrans.rename(columns = {'source_accountno': 'account_no'}, inplace = True)

if len(ctrans) >= 1:

    det = pd.read_sql(f'''
                select
                ifnull(mc.id, mg.id) as client_id,
                ifnull(replace(mc.display_name, 'null', ''), mg.display_name) as client_name,
                sa.account_no
                from m_savings_account sa
                left join m_client mc on mc.id = sa.client_id
                left join m_group mg on mg.id = sa.group_id
                where sa.account_no in ({','.join([str(x) for x in ctrans['account_no'].tolist()])})
                ''', Mifosdb)
    final_trans = pd.merge(ctrans, det, on = 'account_no', how = 'left')
    final_trans['field_127'] = final_trans['field_127'].fillna('0')
    final_trans['error_message'] = final_trans['error_message'].fillna('0')
    final_trans['field_127'] = [i[:256] for i in final_trans['field_127']]
    final_trans['error_message'] = [i[:256] for i in final_trans['error_message']]
    

    final_trans.to_sql('dwh_card_transactions', engine, index = False, if_exists = 'append', chunksize= 10000, method = 'multi')
    
    
else:
    print("No card transactions to export at this time!")
    